In [ ]:
import numpy as np
from datetime import date
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
scratchDir = "/scratch/cslage/starTracker"

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
def  readRubinTV_json(date, camera):
    year = int(date/10000)
    month = int((date - 10000 * year)/100)
    day = int((date - 10000 * year - 100 * month))
    if camera == 'Wide':
        filename = f'/scratch/cslage/starTracker/startracker-wide_{year}-{month:02}-{day:02}.json'
        items = ['MJD wide', 'Exposure Time wide', 'Alt wide', 'Az wide', 'Calculated Az wide', 'Calculated Alt wide', 'Delta Rot Arcsec wide']
    else:
        filename = f'/scratch/cslage/starTracker/startracker_{year}-{month:02}-{day:02}.json'
        items = ['MJD', 'Exposure Time','Alt', 'Az', 'Calculated Az', 'Calculated Alt', 'Delta Rot Arcsec']
    df = pd.read_json(filename)
    df = df.transpose()
    data = {}
    for seqNum in df.index.values.tolist():
        dataList = []
        for name in items:
            item = df.loc[seqNum][name]
            dataList.append(item)
        if ~np.isnan(dataList).any() and len(dataList) == 7:
            data.update({seqNum:dataList})
    return data

## This generates the tpoint input format
### One camera, one date only

In [ ]:
[camera, num] = ['Narrow', 102]
date = 20230309
year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))
path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
outfilename = f"Tpoint_Input_RubinTV_New_{camera}_{date}.dat"
outfile = open(scratchDir + "/" + outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!LSST Main Telescope, 2023 Mar 09 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
data = readRubinTV_json(date, camera)

for key in data.keys():
    [MJD, expTime, el, az, azSolve, elSolve, rot] = data[key]
    rot /= 3600.0
    time = Time(MJD, format='mjd', scale='tai')
    start_time = time - TimeDelta(expTime/2.0, format='sec')
    end_time = time + TimeDelta(expTime/2.0, format='sec')
    azMTM = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                    ['actualPosition'],  start_time.utc, end_time.utc)
    elMTM = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                    ['actualPosition'],  start_time.utc, end_time.utc) 
    azMTMount = np.mean(azMTM.values[:,0])
    azMTMount = azMTMount % 360
    elMTMount = np.mean(elMTM.values[:,0])
    outfile.write(f"{azSolve:.6f}\t{elSolve:.6f}\t{azMTMount:.6f}\t{elMTMount:.6f}\t{rot:.6f}\n")
outfile.write("END\n")
outfile.close()